In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

open a database

In [2]:
string dbPath = @"WedgeFlow_db";
var database = OpenOrCreateDatabase(dbPath);

Creating database 'WedgeFlow_db'.


run WedgeFlow simulation and safe to database

In [3]:
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
                    optiLSDegree: 1,
                    lsDegree: 1,
                    shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
                    MaxIterations: 200,
                    dgDegree: 0,
                    numOfCellsX: 15,
                    numOfCellsY: 10,
                    initialAngle_shockLS: 32,
                    PlotInterval: -1,
                    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
                    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
                    FluxVersion: XESF.Fluxes.FluxVersion.Optimized,
                    agg: 0.4,
                    globalization: ApplicationWithIDT.GlobalizationStrategy.LineSearch
                    );
p.Init(C);
p.RunSolverMode();


rm;
Grid Edge Tags changed.
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             XESF.XESFMain
*  project:                            XESFWedgeFlow_TwoLs
*  session:                            WedgeFlow_TwoLs_p0_xCells15_yCells10_GodunovInterface_iterations200_angle32_lSDegree1_GlobLineSearch_agg0.4_
*  optimization proble

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [4]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);

Error: (1,1): error CS0246: The type or namespace name 'Assembler' could not be found (are you missing a using directive or an assembly reference?)
(1,21): error CS0246: The type or namespace name 'Assembler' could not be found (are you missing a using directive or an assembly reference?)

## Enthalpy error

enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

As we have uniform inflow the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [5]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

Error: (2,31): error CS0103: The name 'si' does not exist in the current context

In [6]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

Error: (1,24): error CS0103: The name 'gamma' does not exist in the current context
(1,32): error CS0103: The name 'gamma' does not exist in the current context
(1,47): error CS0103: The name 'velocityXLeft' does not exist in the current context
(1,61): error CS0103: The name 'velocityXLeft' does not exist in the current context
(6,11): error CS0103: The name 'enthalpy_exact' does not exist in the current context
(2,1): error CS0103: The name 'enthalpy_exact' does not exist in the current context
(3,1): error CS0103: The name 'enthalpy_exact' does not exist in the current context
(4,1): error CS0103: The name 'enthalpy_exact' does not exist in the current context
(5,1): error CS0103: The name 'enthalpy_exact' does not exist in the current context

In [7]:
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Error: (1,22): error CS0103: The name 'si' does not exist in the current context
(1,25): error CS0103: The name 'EEN' does not exist in the current context

In [8]:
p.PlotShadowFields(si);

Error: (1,20): error CS0103: The name 'si' does not exist in the current context